# Анализ и решение систем уравнений материального баланса

[К оглавлению](_contents.ipynb)

![Схема процесса](pics/two_columns_balance.png)

На рисунке показана блок-схема системы разделения, в которой трехкомпонентная смесь разделяется на отдельные компоненты - A, B, C.

В верхней части первой колонны отделяется 20% потока питания (чистый компонент A). Во второй колонне оставшиеся 80% разделяются на отдельные компоненты - B и C.

Требуется:

1. Записать систему уравнений материального баланса по общим массовым расходам потоков
2. Определить число степеней свободы системы
3. Задав (произвольным образом) необходимое количество свободных переменных, найти расходы всех потоков из решения системы материальных балансов.

## Система уравнений материального баланса

Система уравнений материального баланса в неявном виде:

\[
\begin{equation}\label{eq:system}
\begin{cases}
    x_1 - x_2 - x_3 = 0\\
    -0.2 x_1 + x_2 = 0 \\
    -0.8 x_1 + x_3 = 0 \\    
    x_3 - x_4 - x_5 = 0 
\end{cases}
\end{equation}
\]

Количество переменных в системе - 5, количество уравнений - 4

Чтобы определить число независимых уравнений, необходимо найти ранг матрицы коэффициентов системы

В этом примере мы покажем, как решить такую задачу двумя способами:  
- с помощью аналитических преобразований с использованием пакета sympy
- численно, с помощью пакета numpy

Дополнительно покажем, как получить матрицу коэффициентов системы на основе исходных уравнений с помощью SymPy.

## Анализ и решение системы уравнений с помощью символьных вычислений в SymPy

In [1]:
## Подключение пакетов
import numpy as np # Работа с массивами
import sympy as sp # Символьная математика

In [2]:
# Инициализация отображения математических формул:
sp.init_printing()

In [3]:
# Обход проблемы с отображением матриц - определяем функцию для их печати
# 
from IPython.display import  Math
def printMatrix(m):
    """
    Функция для вывода в блокнот матриц SymPy.
    Использование: printMatrix(Матрица)
    """
    return Math(sp.latex(m))

In [4]:
# Определяем переменные для символьных преобразований:
x1, x2, x3, x4, x5 = sp.symbols('x_1 x_2 x_3 x_4 x_5')

In [5]:
# Матрица коэффициентов системы:

# Дроби нужно задавать как рациональные, иначе работать будет неправильно: 0.2 = 2/10 = 1/5
A = sp.Matrix([
        [1, -1, -1, 0, 0],
        [-sp.Rational(1, 5), 1, 0, 0, 0],
        [-sp.Rational(4, 5), 0, 1, 0, 0],
        [0, 0, 1, -1, -1]        
    ])

# Автовывод матриц не работает, если на компьютере не установлен LaTeX. 
# Поэтому используем созданную выше функцию для вывода в графическом режиме:
printMatrix(A)

<IPython.core.display.Math object>

In [6]:
# Или функцию pprint() пакета SymPy для вывода в текстовом режиме:
sp.pprint(A)

⎡ 1    -1  -1  0   0 ⎤
⎢                    ⎥
⎢-1/5  1   0   0   0 ⎥
⎢                    ⎥
⎢-4/5  0   1   0   0 ⎥
⎢                    ⎥
⎣ 0    0   1   -1  -1⎦


In [7]:
# Определяем ранг матрицы с помощью метода rank:
A.rank()

In [8]:
# Определяем ранг матрицы с помощью преобразования к ступенчатому виду:
printMatrix(A.rref()[0])

<IPython.core.display.Math object>

Ранг матрицы определяет число линейно независимых строк. Следовательно, линейно независимы только 3 уравнения системы из четырех.

Число степеней свободы: $F = N - M = 5 - 3 = 2$. Чтобы найти решение, необходимо задать значения двух свободных переменных.

In [9]:
X = sp.Matrix([x1, x2, x3, x4, x5])

In [10]:
# Получаем уравнения системы:
M = A.dot(X)
M

In [11]:
# Пусть заданы расход потока питания и расход компонента C на выходе из системы:

Ms = [eq.subs({x1:100, x5:30}) for eq in M]

# Метод subs делает замену в выражении, у которого он вызывается.
# В качестве аргумента метода надо передать словарь, указывающий, какие переменные на что заменять.
# Поскольку M - это список, содержащий отдельные уравнения системы, нам необходимо сделать замену в каждом уравнении списка.
# Для этого используем списочное выражение: [for i in list]

Ms

In [12]:
# Решаем систему:
sp.solve(Ms)

In [13]:
# Можно обойтись без использования списочного выражения, если преобразовать список уравнений в матрицу.
# Тогда замена будет сделана сразу во всех строках матрицы:

Ms = sp.Matrix(M).subs({x1:100, x5:30})
printMatrix(Ms)

<IPython.core.display.Math object>

In [14]:
sp.solve(Ms)

Расходы потоков составляют:

Поток | Переменная | Расход | Примечание
:-|:-
Поток питания| $x_1$ | 100  |регламентированная
Легкий компонент A | $x_2$ | 20 |  
Кубовый продукт колонны К1 | $x_3$ | 80 |
Легкий компонент B | $x_4$ | 50 |
Тяжелый компонент C | $x_5$ | 30 | регламентированная

## Анализ и решение системы уравнений с помощью численных методов и пакета NumPy

In [15]:
# Матрица коэффициентов системы:
A = np.matrix([
        [1, -1, -1, 0, 0],
        [-0.2, 1, 0, 0, 0],
        [-0.8, 0, 1, 0, 0],
        [0, 0, 1, -1, -1]        
    ])

A

matrix([[ 1. , -1. , -1. ,  0. ,  0. ],
        [-0.2,  1. ,  0. ,  0. ,  0. ],
        [-0.8,  0. ,  1. ,  0. ,  0. ],
        [ 0. ,  0. ,  1. , -1. , -1. ]])

In [16]:
# Ранг матрицы коэффициентов:
np.linalg.matrix_rank(A)

3

Необходимо задать две переменные. Как и ранее, примем $x_1 = 100$ и $x_5 = 30$.

К уравнениям системы $\ref{eq:system}$ добавятся еще два:

\[
    \begin{cases}
    x_1 = 100\\
    x_5 = 30
    \end{cases}
\]

In [17]:
# Свободные члены для исходной системы:
B = np.zeros(4)

# Коэффициенты и свободные члены для дополнительных уравнений
A_dop = np.matrix([
        [1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1]        
    ])

B_dop = np.array([100, 30])

# Матрица системы, дополненная новыми строками:

A_new = np.vstack((A, A_dop)) # для матриц используем функцию vstack(); аргумент - кортеж из склеиваемых матриц
B_new = np.append(B, B_dop) # для массивов используем функцию append()

A_new

matrix([[ 1. , -1. , -1. ,  0. ,  0. ],
        [-0.2,  1. ,  0. ,  0. ,  0. ],
        [-0.8,  0. ,  1. ,  0. ,  0. ],
        [ 0. ,  0. ,  1. , -1. , -1. ],
        [ 1. ,  0. ,  0. ,  0. ,  0. ],
        [ 0. ,  0. ,  0. ,  0. ,  1. ]])

Линейную систему уравнений можно решить с помощью обращения матрицы коэффициентов:

$$ X = A^{-1}\cdot B $$

In [18]:
# Решаем систему:
X = A_new.I.dot(B_new)
X

matrix([[ 100.,   20.,   80.,   50.,   30.]])

## Получение матрицы коэффициентов системы

Матрица коэффициентов может быть получена автоматически, на основе исходных уравнений в системе $\ref{eq:system}$ с помощью SymPy.

In [19]:
# Задаем выражения для каждого уравнения системы:
eq1 = x1 - x2 - x3
eq2 = -sp.Rational(1, 5) * x1 + x2
eq3 = -sp.Rational(4, 5) * x1 + x3
eq4 = x3 - x4 - x5

# Формируем матрицу из уравнений системы:
Eq = sp.Matrix([eq1, eq2, eq3, eq4])

# Формируем список переменных:
X = sp.Matrix([x1, x2, x3, x4, x5])


# Вычисляем Якобиан - матрицу из частных производных уравнений системы по всем переменным.
# Для линейных систем Якобиан совпадает с матрицей коэффициентов A

J = Eq.jacobian(X)
printMatrix(J)

<IPython.core.display.Math object>